In [40]:
import json
import pdb
from collections import defaultdict as ddict
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
import pickle
from pprint import pprint
import sys, os

# Yiheng file for strategies and DA
import inspect
src_file_path = inspect.getfile(lambda: None)
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))), 'yiheng_findfeatures/'))
from dialog_acts_extractor import *
%matplotlib inline

In [3]:
train_data = json.load(open('../../data/negotiation_data/train.json', 'r'))
test_data = json.load(open('../../data/negotiation_data/test.json', 'r'))
dev_data = json.load(open('../../data/negotiation_data/dev.json', 'r'))

In [4]:
print(len(train_data), len(test_data), len(dev_data))
print (len(train_data) + len(test_data) + len(dev_data))

5383 656 643
6682


In [5]:
print(train_data[0].keys())

dict_keys(['uuid', 'scenario', 'scenario_uuid', 'agents', 'outcome', 'events'])


In [5]:
print(train_data[0]['scenario'].keys())

dict_keys(['category', 'uuid', 'post_id', 'kbs', 'attributes', 'intersection'])


In [8]:
pprint(train_data[0]['scenario']['attributes'])

[{'entity': False,
  'multivalued': False,
  'name': 'Role',
  'unique': False,
  'value_type': 'role'},
 {'entity': False,
  'multivalued': False,
  'name': 'Bottomline',
  'unique': False,
  'value_type': 'price'},
 {'entity': False,
  'multivalued': False,
  'name': 'Target',
  'unique': False,
  'value_type': 'price'},
 {'entity': False,
  'multivalued': False,
  'name': 'Title',
  'unique': False,
  'value_type': 'text'},
 {'entity': False,
  'multivalued': False,
  'name': 'Category',
  'unique': False,
  'value_type': 'text'},
 {'entity': False,
  'multivalued': False,
  'name': 'Price',
  'unique': False,
  'value_type': 'price'},
 {'entity': False,
  'multivalued': False,
  'name': 'Images',
  'unique': False,
  'value_type': 'text'},
 {'entity': False,
  'multivalued': False,
  'name': 'Description',
  'unique': False,
  'value_type': 'text'}]


In [9]:
pprint(train_data[0]['events'])

[{'action': 'message',
  'agent': 0,
  'data': 'Hi, not sure if the charger would work for my car. Can you sell it '
          'to me for $5?',
  'start_time': '1496341261.23',
  'time': '1496341272.93'},
 {'action': 'message',
  'agent': 1,
  'data': 'It will work, i have never seen a car without a cigarette lighter '
          'port.\\',
  'start_time': '1496341284.35',
  'time': '1496341307.92'},
 {'action': 'message',
  'agent': 0,
  'data': "Still, can I buy it for $5? I'm on a tight budge",
  'start_time': '1496341318.6',
  'time': '1496341329.15'},
 {'action': 'message',
  'agent': 1,
  'data': 'I think the lowest I would want to go is 8. ',
  'start_time': '1496341336.93',
  'time': '1496341345.99'},
 {'action': 'message',
  'agent': 0,
  'data': "How about $6 and I pick it up myself? It'll save you shipping to "
          'me.',
  'start_time': '1496341364.25',
  'time': '1496341376.38'},
 {'action': 'message',
  'agent': 1,
  'data': '7, and we have a deal.',
  'start_time': 

In [36]:
def print_conv(data):
    '''
    Takes data (eg train_data[1210])
    Prints conversation nicely
    '''
    print ('\n'.join(['AGENT '+str(i['agent']) + ' : ' + i['data'] for i in data['events'] if i['action'] not in ['quit', 
                                                                                                                  'accept', 
                                                                                                                  'offer',
                                                                                                                 'reject']]))

In [37]:
print_conv(train_data[2121]) # this one was rejected

AGENT 1 : Hey there, you're interested in the amp?
AGENT 0 : I am.  Can you tell me how old/new it is?
AGENT 1 : It's only about 6 months, my band was beginning to do gigs but we had a falling out. Now I need to sell it because I'm going back to work. What's your budget?
AGENT 0 : $45 seems a little high considering it is used.  How low do you think you can go?  I was thinking about $25
AGENT 1 : I can't do that, it's barely used and it needs a good owner like you to crank it up to 11. I've only used it at about 7. I can do 35 lowest.
AGENT 0 : Youre at 35, Im at 25.  How about we just meet in the middle at 30 and let this bad boy start rockin'?


# Extract dialog acts / negotiation strategies

In [34]:
train_data[2085]['uuid']

'C_a33de62998364c489f8dafd29a610566'

In [38]:
print_conv(train_data[2085])

AGENT 0 : Hello
AGENT 1 : Hi.
AGENT 0 : So I'm interested in buying the parisian cornice double shelving
AGENT 1 : Great. I am located in Mateca. When would u like to pick up?
AGENT 0 : I can pick up whenever.  How much are you selling for?
AGENT 1 : 600 is what I am asking
AGENT 0 : What condition is the shelving in?
AGENT 1 : Good condition!
AGENT 0 : If I come to pick it up when you want and haul it away, would you be willing to do $525?


In [41]:
#diaacts = extract_acts(train_data[0])
new_data = train_data[2085]
for i in range(len(new_data['events'])):
    if type(new_data['events'][i]['data']) == str:
        new_data['events'][i]['data'] = new_data['events'][i]['data'].lower()
#new_data
seqacts = extract_seq_acts(new_data)

In [42]:
print(seqacts[1])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

In [43]:
my_freq = ddict(int) # id too freq
full_data = train_data + dev_data
for i in range(len(full_data)):
    print (f'{i}/{len(full_data)}', end = '\r')
    for j in range(len(full_data[i]['events'])):
        if type(full_data[i]['events'][j]['data']) == str:
            full_data[i]['events'][j]['data'] = full_data[i]['events'][j]['data'].lower()
    #strats = extract_seq_acts(full_data[i])
    strats = extract_acts(full_data[i])
    strats = strats[1]
    for j in range(len(strats)):
        for idx, val in enumerate(strats[j]):
            my_freq[idx] += val
print ()
print (my_freq)

6025/6026
defaultdict(<class 'int'>, {0: 2798, 1: 10337, 2: 1252, 3: 10582, 4: 1628, 5: 11082, 6: 868, 7: 13466, 8: 8750, 9: 1020, 10: 7992, 11: 758, 12: 17331, 13: 10158, 14: 5298, 15: 3233, 16: 5084, 17: 7153, 18: 2071, 19: 1741, 20: 1589, 21: 1557, 22: 1019, 23: 858, 24: 8948, 25: 0, 26: 2501, 27: 1875, 28: 52, 29: 669, 30: 58, 31: 0, 32: 0, 33: 0, 34: 214, 35: 143, 36: 185, 37: 1107, 38: 45, 39: 3266, 40: 3877})


In [44]:
dialog = train_data[2085]
scene = dialog['scenario']['kbs'][1]['item']['Category'] + ' ' + ' '.join(dialog['scenario']['kbs'][1]['item']['Description']) + ' ' + dialog['scenario']['kbs'][1]['item']['Title']

In [45]:
print(scene)

furniture The late-20th-century postmodern movement embraced centuries of design, combining pared-down modernism with classical detail. Both are reflected in our tower, an exacting reproduction of the French original. Finish/Color - Brown Oak Drifted Item located in Manteca, CA. Original picture is from RH.com Restoration Hardware PARISIAN CORNICE DOUBLE SHELVING


In [11]:
# seqacts[1]

In [46]:
# TAKEN FROM YIHENG'S CODE!!!
recommendation_feature_mapping = {"seller_neg_sentiment":0,"seller_pos_sentiment":1,
                                  "buyer_neg_sentiment":2,"buyer_pos_sentiment":3,
                                  "first_person_plural_count_seller":4,"first_person_singular_count_seller":5,
                                  "first_person_plural_count_buyer":6,"first_person_singular_count_buyer":7,
                                  "third_person_singular_seller":8,"third_person_plural_seller":9,
                                  "third_person_singular_buyer":10,"third_person_plural_buyer":11,
                                  "number_of_diff_dic_pos":12,"number_of_diff_dic_neg":13,
                                  "buyer_propose":14,"seller_propose":15,
                                  "hedge_count_seller":16,"hedge_count_buyer":17,
                                  "assertive_count_seller":18,"assertive_count_buyer":19,
                                  "factive_count_seller":20,"factive_count_buyer":21,
                                  "who_propose":22,"seller_trade_in":23,
                                  "personal_concern_seller":24,"sg_concern":25,
                                  "liwc_certainty":26,"liwc_informal":27,
                                  "politeness_seller_please":28,"politeness_seller_gratitude":29,
                                  "politeness_seller_please_s":30,
                                  "ap_des":31,"ap_pata":32,"ap_infer":33,
                                  "family":34,"friend":35,
                                  "politeness_buyer_please":36,"politeness_buyer_gratitude":37,
                                  "politeness_buyer_please_s":38,
                                  "politeness_seller_greet":39,"politeness_buyer_greet":40}

### Features used

In [47]:
recommendation_feature_reverse_mapping = {v:k for k, v in recommendation_feature_mapping.items()}

In [48]:
def get_strategies(bag_of_strategies):
    '''
    Takes a bag of strategies which is list of (len dialogue) * 41 (features)
    And returns the non one hot encoding
    '''
    return [[recommendation_feature_reverse_mapping[i] for i, val in enumerate(onedia) if val == 1] for onedia in bag_of_strategies]

In [49]:
get_strategies(seqacts[1])

[['politeness_buyer_greet'],
 [],
 [],
 ['seller_pos_sentiment',
  'first_person_singular_count_seller',
  'number_of_diff_dic_pos',
  'number_of_diff_dic_neg',
  'hedge_count_seller'],
 ['first_person_singular_count_buyer'],
 ['first_person_singular_count_seller'],
 [],
 ['seller_pos_sentiment', 'number_of_diff_dic_pos'],
 ['first_person_singular_count_buyer',
  'third_person_singular_buyer',
  'hedge_count_buyer'],
 []]

In [50]:
def addStrategies(data):
    '''
    Takes raw data then adds strategies in each event
    '''
    new_data = data.copy()
    for i in range(len(data)):
        #dia_strategies = get_strategies(extract_seq_acts(data[i])[1])
        dia_strategies = get_strategies(extract_acts(data[i])[1])
        # data[i]['events'][j]['strategies'] = []
        for j in range(len(dia_strategies)):
            #if len(new_data[i]['events']) != len(dia_strategies):
            #    pdb.set_trace()
            new_data[i]['events'][j]['strategies'] = dia_strategies[j]
        if i % 100 == 0: print ("Done : {} / {}".format(i, len(data)), end = '\r')
    print ('\n')
    return new_data

In [51]:
data_w_strategies = {
    'train': addStrategies(train_data),
    'test': addStrategies(test_data),
    'dev': addStrategies(dev_data),}
pickle.dump(data_w_strategies, open('../../data/negotiation_data/data_w_strategies.pkl', 'wb'))

Done : 5300 / 5383

Done : 600 / 656

Done : 600 / 643



# ADD PRICES !!!!!

In [52]:
def getAmountFromSent(sent):
    '''
    Takes a sent and returns the first amount seen in that
    '''
    import re
    import string
    #print (re.findall(r'\d+', sent))
    rr = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", sent)
#    print (rr)
    rr2 = [float(x.translate(str.maketrans('', '', string.punctuation))) for x in rr]
    return min(rr2)

def getAmounts(datapoint):
    '''
    Takes a data point and returns buyer_amt, seller_amt, final_amt and is_accept
    Can update it to include the *first_proposed* price.. to see better how things changed
    '''
    is_accept = False
    buyer_amt, seller_amt, final_amt = -1, -1, -1
    buyer_suggested, seller_suggested = -1, -1
    for j in range(len(datapoint['scenario']['kbs'])):
        if datapoint['scenario']['kbs'][j]['personal']['Role'] == 'buyer': 
            buyer_suggested = datapoint['scenario']['kbs'][j]['personal']['Target']
        if datapoint['scenario']['kbs'][j]['personal']['Role'] == 'seller': 
            seller_suggested = datapoint['scenario']['kbs'][j]['personal']['Target']
    for j in range(len(datapoint['events'])):
        if datapoint['events'][j]['action'] == 'offer':
            final_amt = datapoint['events'][j]['data']['price']
        if datapoint['events'][j]['action'] == 'accept': is_accept = True
    buyer_proposed = False
    seller_proposed = False
    for j in range(len(datapoint['events'])):
        if 'buyer_propose' in datapoint['events'][j]['strategies'] and buyer_proposed == False:
            buyer_proposed = True
            buyer_amt = getAmountFromSent(datapoint['events'][j]['data'])
        if 'seller_propose' in datapoint['events'][j]['strategies'] and seller_proposed == False:
            seller_proposed = True
            seller_amt = getAmountFromSent(datapoint['events'][j]['data'])
#     print_conv(datapoint)
    if is_accept == False:
        final_amt = -1
    buyer_amt = min(buyer_amt, buyer_suggested) # take min for buyer
    seller_amt = max(seller_amt, seller_suggested) # take max for seller
#     print ('buyer suggested : ', buyer_suggested)
#     print ('seller suggested : ', seller_suggested)
#     print ('buyer amt : ', buyer_amt)
#     print ('seller amt: ', seller_amt)
#     print ('final amt : ', final_amt)
#     pdb.set_trace()
    return buyer_amt, seller_amt, final_amt, is_accept

In [53]:
for dtype in data_w_strategies:
    print ('For dtype : ', dtype)
    data = data_w_strategies[dtype]
    total = len(data)
    not_accepts = 0
    not_buyer_propose_in_accepted = 0
    print (len(data))
    for i in range(len(data)):
        datapoint = data[i]
        buyer_amt, seller_amt, final_amt, is_accept = getAmounts(datapoint)
        if final_amt == -1:
            not_accepts += 1
        data[i]['outcome']['buyer_start_amt'] = buyer_amt
        data[i]['outcome']['seller_suggested_amt'] = seller_amt
        data[i]['outcome']['accepted_amt'] = final_amt
    print ('total : ', total)
    print ('not_acc:', not_accepts)

For dtype :  train
5383
total :  5383
not_acc: 1310
For dtype :  test
656
total :  656
not_acc: 185
For dtype :  dev
643
total :  643
not_acc: 182


In [54]:
pickle.dump(data_w_strategies, open('../../data/negotiation_data/data_w_strategies_outcomes.pkl', 'wb'))